In [1]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

In [2]:
# regex pra validar a senha e o nome
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [3]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [9]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["kaiquelima@alunos.utfpr.edu.br"],    # manda um exemplo pela função Field
        description="Endereço de email do usuário",     # descrição do campo
        frozen=True,                                    # só seta, não altera
    )

    # quando printar a senha, mostra *****
    password: SecretStr = Field(
        examples=["kai123"], description="Senha do usuário"
    )

    role: Role = Field(
        default=None, description="Cargo do usuário"
    )

    # adição de um campo obrigatório
    especie : str = Field(
        default=None, description="Espécie do usuário", examples=["Humano", "Robô", "Alienígena"]
    )

    # cargo prático - gênero e idade
    genero: str = Field(
        default=None, description="Gênero do usuário", examples=["Masculino", "Feminino", "Outro"]
    )

    idade: int = Field(
        description="Idade do usuário"
    )

    # =-=-=-=-=-=- FIELD VALIDATOR =-=-=-=-=-=
    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:  # pega o nome e o retorna
        if not VALID_NAME_REGEX.match(v):   # passa pelo filtro do regex
            raise ValueError(               # se erro, retorna mensagem
                "Nome inválido, deve conter apenas letras e ter pelo menos 2 caracteres"
            )
        return v                            # retorna o nome validado

    # role validator
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        # dicionário que mapeia tipos para funções de conversão
        op = {
            int: lambda x: Role(x),      # se for int, converte para role usando o valor
            str: lambda x: Role[x],      # se for string, busca o role pelo nome
            Role: lambda x: x            # se já for role, retorna como está
        }

        try:
            return op[type(v)](v) # pega o tipo do valor recebido e aplica a função correspondente
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    # validação do campo espécie
    @field_validator("especie")
    @classmethod
    def validate_especie(cls, v: str) -> str:
        if not v:
            raise ValueError("Espécie é um campo obrigatório")
        if v not in ["Humano", "Robô", "Alienígena"]:
            raise ValueError("Espécie inválida, deve ser uma das seguintes: Humano, Robô, Alienígena")

        return v

    # validação do campo gênero
    @field_validator("genero")
    @classmethod
    def validate_genero(cls, v: str) -> str:
        if not v:
            raise ValueError("Gênero é um campo obrigatório")
        if v not in ["Masculino", "Feminino", "Outro"]:
            raise ValueError("Gênero inválido, deve ser um dos seguintes: Masculino, Feminino, Outro")

        return v

    # validação do campo idade
    @field_validator("idade")
    @classmethod
    def validate_idade(cls, v:int) -> int:
        if v < 0:
            raise ValueError("Idade não pode ser negativa")
        if v < 18:
            raise ValueError("Idade inválida, deve ser maior ou igual a 18 anos")
        if v > 120:
            raise ValueError("Idade inválida, deve ser menor ou igual a 120 anos")

        return v


    # =-=-=-=-=-=- MODEL VALIDATOR =-=-=-=-=-=
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        # verifica se os campos obrigatórios estão presentes
        if "name" not in v or "password" not in v:
            raise ValueError("Nome e senha são campos obrigatórios")

        # verifica se o nome não está contido na senha (case-insensitive)
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Senha não pode conter o nome do usuário")

        # verifica se o email é a senha
        if v["email"].casefold() in v["password"].casefold():
            raise ValueError("Senha não pode conter o email do usuário")

        # valida a senha usando regex
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Senha inválida, deve conter pelo menos 8 caracteres, uma letra maiúscula, uma letra minúscula e um dígito"
            )

        # hash da senha usando SHA-256 antes de armazenar
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

In [10]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

In [12]:
test_data = dict(
        good_data={
            "name": "Kaique",
            "email": "kaiquelima@alunos.utfpr.edu.br",
            "password": "Password123",
            "role": "Admin",
            "especie": "Humano",
            "genero": "Masculino",
            "idade": 20,
        },

        bad_genero = {
            "name": "Fulano",
            "email": "abacaxi@guairinha.com",
            "password": "Password123",
            "role": "Author",
            "especie": "Humano",
            "genero": "Guarda-roupa", # genero invalido
        },
        bad_idade = {
            "name": "Fulano",
            "email": "hugocalderana@gmail.com",
            "password": "Password123",
            "role": "Author",
            "especie": "Humano",
            "genero": "Masculino",
            "idade": 17,  # idade inválida
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

good_data
name='Kaique' email='kaiquelima@alunos.utfpr.edu.br' password=SecretStr('**********') role=<Role.Admin: 4> especie='Humano' genero='Masculino' idade=20

bad_genero
User is invalid:
2 validation errors for User
genero
  Value error, Gênero inválido, deve ser um dos seguintes: Masculino, Feminino, Outro [type=value_error, input_value='Guarda-roupa', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
idade
  Field required [type=missing, input_value={'name': 'Fulano', 'email...genero': 'Guarda-roupa'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

bad_idade
User is invalid:
1 validation error for User
idade
  Value error, Idade inválida, deve ser maior ou igual a 18 anos [type=value_error, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

missing_data
User is invalid:
1 validation error for User
  Value error, Nome e senha